[link text](https://)<h1> Titanics Dataset Predictions </h1>
<h2> Jose M. Serra Jr. </h2>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import matplotlib.pyplot as mplplt
import numpy as np
import pandas as pd
from scipy.stats import reciprocal
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
from tensorflow.keras.metrics import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.optimizers.schedules import *
import tensorflow.keras.callbacks as tkc
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.manifold import Isomap
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.utils import resample

Here I included standard packages which include TensorFlow machine learning library, as well as the Sci-Kit Learn library for data processing. As well as standard imports such as Pandas for reading data, and numpy for numerial analysis.

In [10]:
file1_train, file2_test = pd.read_csv(r"/content/drive/MyDrive/Titanic/train.csv", delimiter=","),\
pd.read_csv(r"/content/drive/MyDrive/Titanic/test.csv", delimiter= ",")

<p> Now I am going to encode the Sex column into a binary 1s, and 0s output Sex_Binary column.</p>

In [11]:
file1_train = file1_train.drop(columns = ["PassengerId","Name"])
X, y = file1_train.drop("Survived", axis=1), file1_train["Survived"]

In [12]:
file1_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,0,3,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,211536,13.0000,NaN,S
887,1,1,female,19.0,0,0,112053,30.0000,B42,S
888,0,3,female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,1,1,male,26.0,0,0,111369,30.0000,C148,C


In [13]:
objs = X.select_dtypes(["object"])
num  = X.select_dtypes(["number"])

In [14]:
objs.isnull().sum().head()

Sex           0
Ticket        0
Cabin       687
Embarked      2
dtype: int64

In [15]:
num.isnull().sum().head()

Pclass      0
Age       177
SibSp       0
Parch       0
Fare        0
dtype: int64

In [16]:
numerical_features = num.columns
numerical_pipeline = Pipeline(
    steps=[
            ("imputer", SimpleImputer(strategy = 'mean')),
            ("scaler", StandardScaler())
])

In [17]:
for i in range(len(objs.columns)):
    print(objs.columns[i], objs.iloc[:,i].value_counts().unique())

Sex [577 314]
Ticket [7 6 5 4 3 2 1]
Cabin [4 3 2 1]
Embarked [644 168  77]


In [18]:
binary = ["Sex"]
binary_pipeline = Pipeline(steps=[("binary", OneHotEncoder())])

In [19]:
cat1 = ["Ticket", "Cabin", "Embarked"]
catergorical_pipeline = Pipeline(steps=[("imputer", SimpleImputer(strategy = 'most_frequent')), ("ordinal_encoder", OrdinalEncoder()),("scaler", StandardScaler())])

In [20]:
data_preprocessor = ColumnTransformer( [('numerical', numerical_pipeline, numerical_features),
                                      ('binary', binary_pipeline, binary),
                                      ('categorical', catergorical_pipeline, cat1)])

In [21]:
X

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,female,35.0,1,0,113803,53.1000,C123,S
4,3,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,211536,13.0000,NaN,S
887,1,female,19.0,0,0,112053,30.0000,B42,S
888,3,female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,1,male,26.0,0,0,111369,30.0000,C148,C


In [22]:
X = Pipeline(steps=[('processing',data_preprocessor)]).fit_transform(X)

In [23]:
param_distribs = {'learn_rate' : np.array(np.linspace(.1,.9))}

In [24]:
def base_model1(learn_rate = .1):
    input_dim = X.shape[1]
    model =Sequential([
    Dense(200 , input_dim = input_dim, activation= "relu"),
    Dropout(rate=.10),
    Dense(100, activation= "tanh"),
    Dense(1,activation = "sigmoid"),
    ])
    lr_schedule = ExponentialDecay(
    learn_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)
    model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), metrics=["accuracy"])
    return model

In [25]:
checkpoint = [ModelCheckpoint("Titanic.h5", monitor='accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')]
early = EarlyStopping(monitor='accuracy', min_delta=0, patience=10, verbose=1, mode='auto')

In [26]:
NN_clf = KerasClassifier(build_fn=base_model1, epochs=100, verbose=1, callbacks =[checkpoint,early] )#callbacks=[lr_scheduler])
#NN_clf._estimator_type = "classifier"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  """Entry point for launching an IPython kernel.


In [27]:
random_trainor = GridSearchCV(estimator=NN_clf,param_grid=param_distribs, cv=None)
random_trainor.fit(X,(y.values.reshape(-1,1)))

Streaming output truncated to the last 5000 lines.
23/23 [==============================] - 0s 6ms/step - loss: 4.4872 - accuracy: 0.6039
Epoch 16/100
22/23 [===========================>..] - ETA: 0s - loss: 2.1088 - accuracy: 0.6406
Epoch 00016: accuracy did not improve from 0.68680
23/23 [==============================] - 0s 5ms/step - loss: 2.1521 - accuracy: 0.6348
Epoch 17/100
21/23 [==========================>...] - ETA: 0s - loss: 3.0327 - accuracy: 0.6339
Epoch 00017: accuracy did not improve from 0.68680
23/23 [==============================] - 0s 6ms/step - loss: 2.9298 - accuracy: 0.6362
Epoch 18/100
19/23 [=======================>......] - ETA: 0s - loss: 1.3695 - accuracy: 0.6414
Epoch 00018: accuracy did not improve from 0.68680
23/23 [==============================] - 0s 7ms/step - loss: 1.3617 - accuracy: 0.6404
Epoch 19/100
11/23 [=============>................] - ETA: 0s - loss: 1.3401 - accuracy: 0.6534
Epoch 00019: accuracy did not improve from 0.68680
23/23 [======

GridSearchCV(estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f0471e615d0>,
             param_grid={'learn_rate': array([0.1       , 0.11632653, 0.13265306, 0.14897959, 0.16530612,
       0.18163265, 0.19795918, 0.21428571, 0.23061224, 0.24693878,
       0.26326531, 0.27959184, 0.29591837, 0.3122449 , 0.32857143,
       0.34489796, 0.36122449, 0.37755102, 0.39387755, 0.41020408,
       0.42653061, 0.44285714, 0.45918367, 0.4755102 , 0.49183673,
       0.50816327, 0.5244898 , 0.54081633, 0.55714286, 0.57346939,
       0.58979592, 0.60612245, 0.62244898, 0.63877551, 0.65510204,
       0.67142857, 0.6877551 , 0.70408163, 0.72040816, 0.73673469,
       0.75306122, 0.76938776, 0.78571429, 0.80204082, 0.81836735,
       0.83469388, 0.85102041, 0.86734694, 0.88367347, 0.9       ])})

In [28]:
best = random_trainor.best_estimator_.model

In [29]:
best.summary()

Model: "sequential_250"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_750 (Dense)           (None, 200)               2200      
                                                                 
 dropout_250 (Dropout)       (None, 200)               0         
                                                                 
 dense_751 (Dense)           (None, 100)               20100     
                                                                 
 dense_752 (Dense)           (None, 1)                 101       
                                                                 
Total params: 22,401
Trainable params: 22,401
Non-trainable params: 0
_________________________________________________________________


In [30]:
best.save("Titanic.h5")

In [31]:
X_test = data_preprocessor.fit_transform(file2_test)
X_test

array([[ 0.87348191,  0.3349926 , -0.49947002, ..., -0.26949361,
        -0.35149045, -0.47091535],
       [ 0.87348191,  1.32553003,  0.61699237, ...,  0.37293425,
        -0.35149045,  0.70076689],
       [-0.31581919,  2.51417495, -0.49947002, ..., -1.00502694,
        -0.35149045, -0.47091535],
       ...,
       [ 0.87348191,  0.65196458, -0.49947002, ...,  1.53675282,
        -0.35149045,  0.70076689],
       [ 0.87348191,  0.        , -0.49947002, ...,  0.3636237 ,
        -0.35149045,  0.70076689],
       [ 0.87348191,  0.        ,  0.61699237, ..., -0.70708939,
        -0.35149045, -1.64259759]])

In [32]:
PassengerId = file2_test["PassengerId"].to_list()

In [33]:
best.predict(X_test)

array([[0.02051574],
       [0.10947427],
       [0.04215562],
       [0.02051574],
       [0.5188818 ],
       [0.02051574],
       [0.26028338],
       [0.24582049],
       [0.6615104 ],
       [0.02211228],
       [0.02051574],
       [0.03505948],
       [0.8994464 ],
       [0.03505948],
       [0.5417649 ],
       [0.75906533],
       [0.03899688],
       [0.04215562],
       [0.26028338],
       [0.02211228],
       [0.03505948],
       [0.02211228],
       [0.94925964],
       [0.5048506 ],
       [0.7447926 ],
       [0.01699767],
       [0.98913926],
       [0.04215562],
       [0.19235444],
       [0.02051574],
       [0.01703453],
       [0.02491122],
       [0.26028338],
       [0.26028338],
       [0.21941555],
       [0.04215562],
       [0.26028338],
       [0.5188818 ],
       [0.02051574],
       [0.04215562],
       [0.04215562],
       [0.4458508 ],
       [0.02051574],
       [0.83760035],
       [0.8994464 ],
       [0.02051574],
       [0.03505948],
       [0.020

In [34]:
final_pred = (best.predict(X_test) > 0.5).astype("int32").flatten()

In [35]:
final_pred

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [36]:
#d = {"PassengerId":PassengerId, "Survived":final_pred}
#pd.DataFrame(data=d,index=None, columns= ["PassengerId","Survived"]).to_csv("12232021.csv",index=False, header=1)